# Example creating a Bitcoin signal

In this blog we will be extending the work done previously and creating a bitcoin signal for a future 10 minute price. We will be calling two models one which forecasts the price and one which shows a forecast of the trend. These models we will call from AWS Marketplace. What you will need to run this article:-

* an AWS account
* subscibe to the models
* reasonable Python skills

## Let us remember what we did in the last article

In the last article we wrote a function called Get_Smush that retrieved a large series of data we will need that again, you can download the code from the GIT repo. First we'll import a few packages some which are related to AWS and Sagemaker which is the delivery method.

In [28]:
import pandas as pd
import json
import csv
import ssl 
import requests
import sagemaker
import os
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [4]:
def GetData_Smush():
    ## get data from cryptodownload 
    data_location = 'https://www.cryptodatadownload.com/cdd/Bitstamp_BTCUSD_minute.csv'
    ssl._create_default_https_context = ssl._create_unverified_context
    hist_data = pd.read_csv(data_location, skiprows=1)

    ## get latest data from bitstamp public API
    json_location=f'https://www.bitstamp.net/api/v2/ohlc/btcusd/?limit=1000&step=60'
    response =  requests.get(json_location)
    ld = json.loads(response.text)
    df = ld['data']['ohlc']
    latest_data = pd.DataFrame(df, columns=['high', 'timestamp', 'volume', 'low', 'close', 'open']) 

    latest_data['date'] = pd.to_datetime(latest_data['timestamp'], unit='s')
    latest_data['volume'] = pd.to_numeric(latest_data['volume'], errors='coerce')
    latest_data['close'] = pd.to_numeric(latest_data['close'], errors='coerce')
    latest_data['Volume_Currency'] = latest_data['volume']*latest_data['close'] 

    latest_data.columns = ['High', 'unix', 'Volume_BTC', 'Low', 'Close', 'Open', 'date', 'Volume_Currency']

            
    # re-arrange data
    dp1 = hist_data.head(20160)
    dp1 = dp1.drop('symbol', axis = 1)
    dp1.columns = ['unix','date','Open','High','Low','Close','Volume_BTC','Volume_Currency']
    row1 = latest_data.head(100)


    detail = int(row1.iloc[0]['unix'])
    dat = dp1.loc[dp1['unix'] < detail]
    dat = dat.iloc[::-1]
    latest_data = latest_data[['unix','date','Open','High','Low', 'Close', 'Volume_BTC', 'Volume_Currency']]



    dat= dat.append(latest_data)
    dat = dat.tail(20160)
    dat['High'] = pd.to_numeric(dat['High'], errors='coerce')
    dat['Low'] = pd.to_numeric(dat['Low'], errors='coerce')
    dat['Weighted_Price'] = (dat['High']+dat['Low'])/2
    
    dat = dat[['unix','date','Open','High','Low', 'Close', 'Weighted_Price','Volume_BTC', 'Volume_Currency']]

    ## return data
    return dat


## We create a new function

We create a function that will also call only the latest datapoint and add it on to our previous series we do this so that we do not hit the limits of the exchange website for calling free data.

In [5]:
# get it for one data point
def GetOnePoint(smush):
    json_location=f'https://www.bitstamp.net/api/v2/ohlc/btcusd/?limit=1&step=60'
    response =  requests.get(json_location)
    ld = json.loads(response.text)
    df = ld['data']['ohlc']
    latest_data = pd.DataFrame(df, columns=['high', 'timestamp', 'volume', 'low', 'close', 'open']) 
    latest_data['date'] = pd.to_datetime(latest_data['timestamp'], unit='s')
    latest_data['volume'] = pd.to_numeric(latest_data['volume'], errors='coerce')
    latest_data['close'] = pd.to_numeric(latest_data['close'], errors='coerce')
    latest_data['Volume_Currency'] = latest_data['volume']*latest_data['close']
    latest_data.columns = ['High', 'unix', 'Volume_BTC', 'Low', 'Close', 'Open', 'date', 'Volume_Currency']
    latest_data['High'] = pd.to_numeric(latest_data['High'], errors='coerce')
    latest_data['Low'] = pd.to_numeric(latest_data['Low'], errors='coerce')
 
    latest_data['Weighted_Price'] = (latest_data['High']+latest_data['Low'])/2
    latest_data.columns = ['High', 'unix', 'Volume_BTC', 'Low', 'Close', 'Open', 'date', 'Volume_Currency','Weighted_Price']
    latest_data = latest_data[['unix','date','Open','High','Low', 'Close','Weighted_Price', 'Volume_BTC', 'Volume_Currency']]
    
    dat= smush.append(latest_data)
    dat = dat.tail(20160)
    return dat

## Create the models 

Now lets create the models:-

* The first model will predict the trend. In this case we will use the model predict10trend.
* The second model will predict the actual price. This model will be BitcoinPredict10.

We will then combine these models to give a future signal that can be used to take a trading position.

In [15]:
#The trend model predict10trend

content_type='text/csv'
model_name='predict10trend-copy-04-24-copy-04-24'
real_time_inference_instance_type='ml.m5.large'

# Please use the appropriate ARN obtained after subscribing to the model to define 'model_package_arn'
model_package_arn = 'arn:aws:sagemaker:eu-west-x:xxxxxxx:model-package/predict10trend-copy-04-24-copy-04-24'

from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [16]:
 trend = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------------!

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
#The prediction model bitcoinpredict10

content_type='text/csv'
model_name='bitcoinpredict10'
real_time_inference_instance_type='ml.m5.large'

# Please use the appropriate ARN obtained after subscribing to the model to define 'model_package_arn'
model_package_arn = 'arn:aws:sagemaker:eu-west-x:xxxxxxx:model-package/bitcoinpredict10'

from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [22]:
 predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------------!

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Let's create a buy/sell signal

First we will create a function that will tell us when the predicted price will be above the trend accounting for a strike percentage in this case 5% to capture price moves throughout the day that are above minimum 0.5% (most exchanges trading fee). Finally the ifelse statements will tell us if we should possibly buy/sell. Please remember this is an example.

In [23]:
def signal(pre, tren, strike):
    strikepriceplus = 1+strike
    strikepriceneg = 1-strike
    
    if (tren*strikepriceplus < pre):
        passback = 'sell'
    elif (tren*strikepriceneg > pre):
        passback = 'buy'
    else:
        passback = 'hold'
    
    
    return passback

In [35]:
# get data for model
capture = GetData_Smush()
# data wrangling
capture = capture.drop(['unix','date'], axis=1)
capture = capture[['High','Open','Low','Close','Weighted_Price','Volume_BTC','Volume_Currency']]
capture2 = capture.to_csv(index=False)

# call prediction model
prd = predictor.predict(capture2)

# call trend model
trd = trend.predict(capture2)

In [30]:
def getvaluecrecord(prediction):
    s=str(prediction,'utf-8')
    prediction =s.split(":")
    prediction = prediction[1]
    prediction = prediction.split(",")
    prediction = prediction[0]
    prediction = float(prediction)
    return prediction


In [36]:
prd = getvaluecrecord(prd)
trd =getvaluecrecord(trd)

In [37]:
print('trend = %f and prediction= %f'%(trd,prd))

trend = 57927.529700 and prediction= 58958.154700


In [38]:
sig = signal(prd, trd, 0.05)

In [39]:
sig

'hold'

In [40]:
newdata = GetOnePoint(capture)
newdata = newdata.drop(['unix','date'], axis=1)
newdata = newdata[['High','Open','Low','Close','Weighted_Price','Volume_BTC','Volume_Currency']]
newdata = newdata.to_csv(index=False)

In [41]:
# call prediction model
prd = predictor.predict(newdata)
# call trend model
trd = trend.predict(newdata)

prd = getvaluecrecord(prd)
trd =getvaluecrecord(trd)

sig = signal(prd, trd, 0.05)
sig

'hold'

In [42]:
print('trend = %f and prediction= %f'%(trd,prd))

trend = 57936.215500 and prediction= 58918.503400
